<a href="https://colab.research.google.com/github/tousifo/ml_notebooks/blob/main/MedMNIST_QNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Snippet 1 (Third User Correction): Data Pipeline with Augmentations

# --- Install ---
!pip install medmnist --quiet

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import medmnist

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Histopathology-Friendly Augmentations ---
aug_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.03),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]),
])

eval_tfm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]),
])

# --- Data Loading ---
DATASET_NAMES = ['PathMNIST', 'DermaMNIST', 'BloodMNIST']
datasets, dataloaders = {}, {}
print("\n--- Loading Datasets (with Augmentations) ---")

for name in DATASET_NAMES:
    info = medmnist.INFO[name.lower()]
    DataClass = getattr(medmnist, info['python_class'])
    n_channels = info.get('n_channels', 3)

    # Use the appropriate transform for each split
    train_ds = DataClass(split='train', transform=aug_train, download=True)
    val_ds   = DataClass(split='val',   transform=eval_tfm, download=True)
    test_ds  = DataClass(split='test',  transform=eval_tfm, download=True)

    datasets[name] = {'train': train_ds, 'val': val_ds, 'test': test_ds}

    dataloaders[name] = {
        'train': DataLoader(train_ds, batch_size=64, shuffle=True,  num_workers=2, pin_memory=True),
        'val':   DataLoader(val_ds,   batch_size=128, shuffle=False, num_workers=2, pin_memory=True),
        'test':  DataLoader(test_ds,  batch_size=128, shuffle=False, num_workers=2, pin_memory=True),
    }

    print(f"{name}: train={len(train_ds)}, val={len(val_ds)}, test={len(test_ds)}, "
          f"classes={len(train_ds.info['label'])}, channels={n_channels}")

print("\nAll datasets with advanced augmentations are ready.\n")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 4.6 MB/s eta 0:00:00
Using device: cuda

--- Loading Datasets (with Augmentations) ---


100%|██████████| 206M/206M [02:47<00:00, 1.22MB/s]


PathMNIST: train=89996, val=10004, test=7180, classes=9, channels=3


100%|██████████| 19.7M/19.7M [00:17<00:00, 1.12MB/s]


DermaMNIST: train=7007, val=1003, test=2005, classes=7, channels=3


100%|██████████| 35.5M/35.5M [00:34<00:00, 1.03MB/s]


BloodMNIST: train=11959, val=1712, test=3421, classes=8, channels=3

All datasets with advanced augmentations are ready.



In [2]:
# Snippet 2 (Third User Correction): The Trainable Quantum Model

# --- Install ---
!pip install pennylane --quiet

import torch
import torch.nn as nn
import pennylane as qml

# --- Trainable QLayer with data re-uploading & dual-basis readout ---
n_qubits = 8
layers = 2  # re-uploading depth
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface='torch', diff_method='backprop')
def trainable_qnode(inputs, theta_rz, theta_ry):
    # inputs: (B, 8) -- TorchLayer handles batching
    # First embedding
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')

    for l in range(layers):
        # Variational block
        for i in range(n_qubits):
            qml.RZ(theta_rz[l, i], wires=i)
            qml.RY(theta_ry[l, i], wires=i)
        # Ring entanglement
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i+1])
        qml.CNOT(wires=[n_qubits-1, 0])

        # Re-upload the same 8 features (gives richer gradients)
        qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')

    # Read out in two Pauli bases -> 16 quantum features
    z_feats = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    x_feats = [qml.expval(qml.PauliX(i)) for i in range(n_qubits)]
    return z_feats + x_feats

weight_shapes = {
    "theta_rz": (layers, n_qubits),
    "theta_ry": (layers, n_qubits),
}

class QTFClassifier(nn.Module):
    def __init__(self, q_feature_map, weight_shapes, n_classes=9, in_channels=3):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, n_qubits)   # -> 8 features
        )

        # Trainable quantum layer (outputs 16-dim: Z and X on 8 qubits)
        self.q_layer = qml.qnn.TorchLayer(q_feature_map, weight_shapes=weight_shapes)

        # Final head now expects 16 inputs
        self.classifier = nn.Sequential(
            nn.Linear(2 * n_qubits, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, n_classes)
        )

    def forward(self, x):
        feats = self.feature_extractor(x)         # (B, 8), float32
        qfeats = self.q_layer(feats)              # (B, 16)
        qfeats = qfeats.to(feats.dtype)           # keep dtypes consistent
        logits = self.classifier(qfeats)
        return logits

print("Trainable QTF model ready (re-uploading + dual-basis readout).")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 62.7 MB/s eta 0:00:00
Trainable QTF model ready (re-uploading + dual-basis readout).


In [3]:
# Snippet 3 (Third User Correction): The Advanced Training Loop

from tqdm import tqdm
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingLR

# --- Setup ---
dataset_name = 'PathMNIST'
info = medmnist.INFO[dataset_name.lower()]
in_channels = info.get('n_channels', 3)
n_classes = len(info['label'])

# --- Build Model ---
# This instantiates our final, trainable QTFClassifier
model = QTFClassifier(q_feature_map=trainable_qnode,
                      weight_shapes=weight_shapes,
                      n_classes=n_classes,
                      in_channels=in_channels).to(device)

# --- Optimizer, Loss, and Scheduler ---
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-4)

# Label smoothing (with a fallback for older torch versions)
try:
    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.05)
    print("Using CrossEntropyLoss with label smoothing.")
except TypeError:
    loss_fn = nn.CrossEntropyLoss()
    print("Using standard CrossEntropyLoss (label smoothing not supported).")

# Cosine annealing scheduler for robust convergence
scheduler = CosineAnnealingLR(optimizer, T_max=25, eta_min=1e-5)

# --- DataLoaders ---
train_loader = dataloaders[dataset_name]['train']
val_loader = dataloaders[dataset_name]['val']

def labels_to_indices(y):
    if y.ndim == 2 and y.size(1) > 1:
        return y.argmax(dim=1)
    return y.view(-1).long()

# --- Full Training & Validation Loop ---
num_epochs = 25
best_val = 0.0
print(f"\nStarting final training run on {dataset_name} for {num_epochs} epochs...")

for epoch in range(1, num_epochs + 1):
    # ---- Train ----
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs} [train]"):
        x = x.to(device, non_blocking=True)
        y = labels_to_indices(y).to(device, non_blocking=True)

        optimizer.zero_grad()
        logits = model(x)
        loss = loss_fn(logits, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        train_correct += (logits.argmax(1) == y).sum().item()
        train_total += x.size(0)
    tr_loss = train_loss / train_total
    tr_acc = train_correct / train_total

    # ---- Validate ----
    model.eval()
    val_correct, val_total, val_loss = 0, 0, 0
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc=f"Epoch {epoch}/{num_epochs} [val]"):
            x = x.to(device, non_blocking=True)
            y = labels_to_indices(y).to(device, non_blocking=True)
            logits = model(x)
            loss = loss_fn(logits, y)
            val_loss += loss.item() * x.size(0)
            val_correct += (logits.argmax(1) == y).sum().item()
            val_total += x.size(0)
    v_loss = val_loss / val_total
    v_acc = val_correct / val_total

    print(f"Epoch {epoch}: train_loss={tr_loss:.4f} train_acc={tr_acc:.4f} "
          f"val_loss={v_loss:.4f} val_acc={v_acc:.4f}")

    # Step the scheduler
    scheduler.step()

    # Save the best model
    if v_acc > best_val:
        best_val = v_acc
        best_state = {k: v.cpu() for k, v in model.state_dict().items()}

print(f"\nTraining complete. Best validation accuracy: {best_val:.4f}")

# Load the best performing model state for our XAI phase
if 'best_state' in locals():
    model.load_state_dict(best_state)
    print("Loaded best model weights. Ready to begin Phase 2: Explainable AI.")

Using CrossEntropyLoss with label smoothing.

Starting final training run on PathMNIST for 25 epochs...


Epoch 1/25 [val]: 100%|██████████| 79/79 [00:06<00:00, 11.53it/s]


Epoch 1: train_loss=2.0873 train_acc=0.1922 val_loss=1.9960 val_acc=0.2219


Epoch 2/25 [val]: 100%|██████████| 79/79 [00:06<00:00, 11.59it/s]


Epoch 2: train_loss=2.0149 train_acc=0.2249 val_loss=2.0113 val_acc=0.2382


...

Epoch 23: train_loss=0.8041 train_acc=0.7898 val_loss=0.7685 val_acc=0.7837


Epoch 24/25 [val]: 100%|██████████| 79/79 [00:06<00:00, 11.61it/s]


Epoch 24: train_loss=0.7977 train_acc=0.7923 val_loss=0.7431 val_acc=0.8006


Epoch 25/25 [val]: 100%|██████████| 79/79 [00:07<00:00, 11.14it/s]

Epoch 25: train_loss=0.7950 train_acc=0.7936 val_loss=0.7484 val_acc=0.7970

Training complete. Best validation accuracy: 0.8028
Loaded best model weights. Ready to begin Phase 2: Explainable AI.
